In [11]:
import re

import pandas as pd

In [ ]:
# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')

In [16]:
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
df.head(3)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,category,category:confidence,category_gold,id,screenname,text
0,662822308,False,finalized,3,2/18/15 4:31,Information,1.0,NaN,4.365280e+17,Barclays,Barclays CEO stresses the importance of regula...
1,662822309,False,finalized,3,2/18/15 13:55,Information,1.0,NaN,3.860130e+17,Barclays,Barclays announces result of Rights Issue http...
2,662822310,False,finalized,3,2/18/15 8:43,Information,1.0,NaN,3.795800e+17,Barclays,Barclays publishes its prospectus for its å£5....


In [4]:
df['category'].value_counts()

Information    2129
Action          724
Dialogue        226
Exclude          39
Name: category, dtype: int64

In [5]:
df['category:confidence'].value_counts(dropna= False)

1.0000    2430
0.6614      35
0.6643      33
0.6747      32
0.6775      29
          ... 
0.7202       1
0.8860       1
0.8570       1
0.6568       1
0.9041       1
Name: category:confidence, Length: 194, dtype: int64

Only keep the rows with `confidence` of 100%. Also, remove the `category` of `Exclude`, as there are very few of them anyway.

### 0. Load the dataset

In [6]:
def load_data():
    df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
    df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
    X = df.text.values
    y = df.category.values
    return X, y

In [7]:
X, y = load_data()

In [8]:
X[0]

'Barclays CEO stresses the importance of regulatory and cultural reform in financial services at Brussels conference  http://t.co/Ge9Lp7hpyG'

### 1. Clean & Tokenize

Replace the urls in the messages with `urlplaceholder`:

In [9]:
# RegEx for url pattern
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [10]:
msg = X[0]
msg

'Barclays CEO stresses the importance of regulatory and cultural reform in financial services at Brussels conference  http://t.co/Ge9Lp7hpyG'

In [12]:
re.findall(url_regex, msg)

['http://t.co/Ge9Lp7hpyG']

In [13]:
url = 'http://t.co/Ge9Lp7hpyG'
msg.replace(url, 'urlplaceholder')

'Barclays CEO stresses the importance of regulatory and cultural reform in financial services at Brussels conference  urlplaceholder'

In [20]:
def tokenize(txt):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, txt)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        txt = txt.replace(url, 'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(txt.lower())
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer
    # Lemmatize and remove the white spaces - leading & trailing
    tokens_clean = [lemmatizer().lemmatize(token.strip()) for token in tokens]

    return tokens_clean

In [24]:
# test out function
X, y = load_data()
for msg in X[:5]:
    tokens = tokenize(msg)
    print(msg)
    print('=======> ', tokens, '\n')
    print('=' * 72)

Barclays CEO stresses the importance of regulatory and cultural reform in financial services at Brussels conference  http://t.co/Ge9Lp7hpyG
=======>  ['barclays', 'ceo', 'stress', 'the', 'importance', 'of', 'regulatory', 'and', 'cultural', 'reform', 'in', 'financial', 'service', 'at', 'brussels', 'conference', 'urlplaceholder'] 

Barclays announces result of Rights Issue http://t.co/LbIqqh3wwG
=======>  ['barclays', 'announces', 'result', 'of', 'right', 'issue', 'urlplaceholder'] 

Barclays publishes its prospectus for its å£5.8bn Rights Issue: http://t.co/YZk24iE8G6
=======>  ['barclays', 'publishes', 'it', 'prospectus', 'for', 'it', 'å£5.8bn', 'right', 'issue', ':', 'urlplaceholder'] 

Barclays Group Finance Director Chris Lucas is to step down at the end of the week due to ill health http://t.co/nkuHoAfnSD
=======>  ['barclays', 'group', 'finance', 'director', 'chris', 'lucas', 'is', 'to', 'step', 'down', 'at', 'the', 'end', 'of', 'the', 'week', 'due', 'to', 'ill', 'health', 'urlpla